In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
final_df = (
    orders
        .merge(users, on="user_id", how="left")
        .merge(restaurants, on="restaurant_id", how="left")
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
# Drop the restaurant name from orders file (keep master data version)
final_df = final_df.drop(columns=["restaurant_name_x"])

# Rename SQL version properly
final_df = final_df.rename(columns={"restaurant_name_y": "restaurant_name"})

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name,cuisine,rating
0,1,2508,450,18-02-2023,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
final_df.shape



(10000, 11)

In [8]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [9]:
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].sum().sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [16]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [17]:
(final_df.groupby("user_id")["total_amount"].sum() > 1000).sum()

2544

In [18]:
final_df.groupby(
pd.cut(final_df["rating"], bins=[3,3.5,4,4.5,5])
)["total_amount"].sum()

rating
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
(4.0, 4.5]    1960326.26
(4.5, 5.0]    2197030.75
Name: total_amount, dtype: float64

In [20]:
(
    final_df[final_df["membership"]=="Gold"]
        .groupby("city")["total_amount"]
        .mean()
        .sort_values(ascending=False)
)


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [21]:
final_df.groupby("cuisine").agg(
restaurants=("restaurant_id","nunique"),
revenue=("total_amount","sum")
).sort_values(by="restaurants")

,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [22]:
(final_df["membership"].eq("Gold").mean()*100).round()

50.0

In [24]:
(
    final_df
        .groupby("restaurant_name")
        .agg(
            count=("order_id","count"),
            avg=("total_amount","mean")
        )
        .query("count < 20")
        .sort_values(by="avg", ascending=False)
)


,count,avg
restaurant_name,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [29]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")
final_df[["order_date","quarter"]].head()


,order_date,quarter
0,2023-02-18,2023Q1
1,2023-01-18,2023Q1
2,2023-07-15,2023Q3
3,2023-04-10,2023Q2
4,2023-12-25,2023Q4


In [30]:
(
    final_df
        .groupby("quarter")["total_amount"]
        .sum()
        .sort_values(ascending=False)
)


quarter
2023Q1    2024039.06
2023Q3    2009284.71
2023Q2    1986691.93
2023Q4    1974406.92
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [31]:
# numerical questions 
#1
final_df[final_df["membership"]=="Gold"].shape[0]



4987

In [32]:
# 2Total revenue from Hyderabad (rounded)
round(final_df[final_df["city"]=="Hyderabad"]["total_amount"].sum())

1889367

In [33]:
#3 Distinct users
final_df["user_id"].nunique()

2883

In [34]:
# 4 Average order value for Gold members (2 decimals)
round(final_df[final_df["membership"]=="Gold"]["total_amount"].mean(), 2)

797.15

In [35]:
# 5 Orders where rating ≥ 4.5
final_df[final_df["rating"] >= 4.5].shape[0]

3374

In [36]:
#6 Orders in top-revenue Gold city
gold_city = (
    final_df[final_df["membership"]=="Gold"]
        .groupby("city")["total_amount"]
        .sum()
        .idxmax()
)

final_df[
    (final_df["membership"]=="Gold") &
    (final_df["city"]==gold_city)
].shape[0]


1337

In [44]:
# fill ups
# 1 Column used to join orders.csv and users.json
orders.columns.intersection(users.columns)



Index(['user_id'], dtype='object')

In [38]:
# 2 Dataset containing cuisine and rating info format
"restaurants.sql (SQL format)"



'restaurants.sql (SQL format)'

In [40]:
# 3Total number of rows in final merged dataset


final_df.shape[0]

10000

In [41]:
#4 If user has no match → merged values become



final_df.isnull().sum()

order_id           0
user_id            0
restaurant_id      0
order_date         0
total_amount       0
name               0
city               0
membership         0
restaurant_name    0
cuisine            0
rating             0
quarter            0
dtype: int64

In [43]:
# 6 membership column originates from which file



users.columns

Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [45]:
'restaurant_id' in final_df.columns

True